<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generate-atomistic-model" data-toc-modified-id="Generate-atomistic-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate atomistic model</a></span></li><li><span><a href="#Convert-the-model-to-3SPN" data-toc-modified-id="Convert-the-model-to-3SPN-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Convert the model to 3SPN</a></span></li></ul></div>

## Generate atomistic model

In [1]:
import sys

#Include the module in the python path
sys.path.insert(0, '../../../open3SPN2')
import ff3SPN2

#Reload the module (only necessary if the module has recently changed)
import importlib
importlib.reload(ff3SPN2)

#The default type is B_curved (3SPN2.C) but it is also possible to change the type to A or B (3SPN2)
dna=ff3SPN2.DNA.fromSequence('ATACAAAGGTGCGAGGTTTCTATGCTCCCACG',dna_type='B_curved')
dna.computeTopology()
#Create the system
s=ff3SPN2.System(dna)

#Clear Forces
s.clearForces()
#s.top.topology.setUnitCellDimensions([50,50,50])
#s.setDefaultPeriodicBoxVectors([100,0,0],[0,100,0],[0,0,100])

#Add 3SPN2 forces
s.add3SPN2forces()


/home/cab22/Programs/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Periodic boundary conditions not defined, system would be non periodic
Bond
Angle
Stacking
Dihedral
BasePair
CrossStacking
Exclusion
Electrostatics
77.8 0.974998211
0.94856634121177 nm 3.5975908035210326e-70 mol C**2 F/m


In [2]:
#Set up the simulation
import simtk.openmm
import simtk.openmm.app
import simtk.unit
temperature=300 * simtk.openmm.unit.kelvin
platform_name='OpenCL'

integrator = simtk.openmm.LangevinIntegrator(temperature, 5 / simtk.openmm.unit.picosecond, 2 * simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(s.top,s, integrator,platform)
simulation.context.setPositions(s.coord.getPositions())
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)
#simulation.reporters.append(simtk.openmm.app.PDBReporter(f'output.pdb', 1000),)
simulation.reporters.append(simtk.openmm.app.DCDReporter(f'output.dcd', 1000),)
#sim_out=open('sim_out.txt','w+')
simulation.reporters.append(simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,potentialEnergy=True, temperature=True,separator='\t',))
simulation.reporters.append(simtk.openmm.app.StateDataReporter('sim.log', 1000, step=True,time=True,totalEnergy=True,
                                              kineticEnergy=True,potentialEnergy=True, temperature=True))

-1393.5015869140625


In [3]:
#Obtain first energies
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',energy)
energies = {}
for f, i in s.force_groups.items():
    energies[f] = simulation.context.getState(getEnergy=True, groups=2**i).getPotentialEnergy().value_in_unit(energy_unit)

for idd in s.force_groups.keys():
    print(idd,s.force_groups[idd],energies[idd] /4.184)

TotalEnergy -1393.5015869140625
Bond 6 4.8611113655745505e-06
Angle 7 0.00051294794452179
Stacking 8 -142.54506915294874
Dihedral 9 -200.7647745923832
BasePair 10 0.8711880981124381
CrossStacking 11 -1.31967509909752
Exclusion 12 0.028444126419309903
Electrostatics 13 10.674528592403258


In [4]:
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(10000)
#Obtain first energies
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',energy)
energies = {}
for f, i in s.force_groups.items():
    energies[f] = simulation.context.getState(getEnergy=True, groups=2**i).getPotentialEnergy().value_in_unit(energy_unit)

for idd in s.force_groups.keys():
    print(idd,s.force_groups[idd],energies[idd] /4.184)

#"Step"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"
1000	2.0000000000000013	-978.7445068359375	298.8436501943973
2000	3.999999999999781	-966.2251586914062	282.4174682602728
3000	5.999999999999561	-958.535888671875	286.1488964753857
4000	7.999999999999341	-960.2212524414062	342.39440859170656
5000	10.000000000000009	-995.435791015625	318.5110065206065
6000	12.000000000000677	-1026.7757568359375	290.05133035185526
7000	14.000000000001345	-985.9140014648438	309.1652887410555
8000	16.00000000000201	-949.3331298828125	345.32774514691414
9000	18.000000000000902	-925.3999633789062	315.83702683866795
10000	19.999999999999794	-932.562255859375	349.8167368552949
TotalEnergy -932.562255859375
Bond 6 26.477171859595334
Angle 7 71.93521970088806
Stacking 8 -151.3200107085773
Dihedral 9 -178.00442347335084
BasePair 10 0.507209656571339
CrossStacking 11 -3.141594206625827
Exclusion 12 0.18693646327031502
Electrostatics 13 10.471741510394882
